In [2]:
import glob
import os
import re
import pandas as pd
import numpy as np
from collections import defaultdict

## Extract mean best scores among all random seeds

In [6]:
# input_dir = './results/randaug_tests/k3'
input_dir = r'C:\Users\hyeon\Desktop\result\nl100_2_fixmatch\result_way2_nl100'
out_dir = './results/extracted_for_best'
out_fname = 'randaug_tests.csv'
if not os.path.isdir(out_dir):
    os.makedirs(out_dir)

In [13]:
fm_cols

['fixaug_nl100_2-nl100-m1-b16-r0',
 'fixaug_nl100_2-nl100-m1-b16-r1',
 'fixaug_nl100_2-nl100-m2-b16-r0',
 'fixaug_nl100_2-nl100-m2-b16-r1',
 'fixaug_nl100_2-nl100-m3-b16-r0',
 'fixaug_nl100_2-nl100-m3-b16-r1']

In [12]:
ori_data_paths = glob.glob(f'{input_dir}/*.txt')

fm_cols = []
data_paths_dict = DefaultOrderedDict(list)
for ori_data_path in ori_data_paths:
    cond = '-'.join(ori_data_path.split('-')[:5])
    # cond = cond.split('/')[-1].replace('nl50', 'nl050')
    cond = cond.split('\\')[-1].replace('nl50', 'nl050')
    if cond not in fm_cols:
        fm_cols.append(cond)
    data_paths_dict[cond].append(ori_data_path)

fm_cols.sort()
len(data_paths_dict)

6

In [14]:
p = re.compile('1[.]\d{4}|0[.]\d{4}') #[0.숫자4개]

cols = ('acc-all', 'ppv-all', 'ppv-covid', 'ppv-pneumonia', 'ppv-normal',
        'recall-all', 'recall-covid', 'recall-pneumonia', 'recall-normal',
        'f1-all', 'f1-covid', 'f1-pneumonia', 'f1-normal')
metrics_dfs = {}
for col, data_paths in data_paths_dict.items(): # 데이터 경로에서 txt파일을 하나씩 불러온다.
    metrics_dfs[col] = pd.DataFrame([], columns=cols)
    for ori_data_path in data_paths:
        with open(ori_data_path, 'r') as f: 
            all_data = f.read() # 전체 txt line을 읽어온다.
            all_data = all_data.split('\n') # 띄어쓰기 기준으로 나눔
            if 'baseline' in ori_data_path:
#                 candidates = ((327, 332), (662, 667), (997, 1002),
#                               (1332, 1337), (1667, 1672))
                candidates = ((327, 332), (662, 667), (997, 1002))
#                 candidates = ((647, 652), (1302, 1307), (1957, 1962))
            else:
#                 candidates = ((347, 352), (702, 707), (1057, 1062),
#                               (1412, 1417), (1767, 1772))
                candidates = ((347, 352), (702, 707), (1057, 1062))
            metrics = []
            for s_idx, e_idx in candidates:
                metric_line = all_data[s_idx:e_idx] # hard coding.. best result 부분
                metrics.append(list(map(float, p.findall(str(metric_line)))))# 정규표현식에 맞는것만 뽑기

            metrics_dfs[col] = pd.concat((metrics_dfs[col],
                                         pd.DataFrame(metrics, columns=cols))).reset_index(drop=True)

final_metrics_df = pd.DataFrame([], columns=cols)
for fm_col in fm_cols:
    final_metrics = []
    for col in cols:
        best_metrics = metrics_dfs[fm_col][col].to_numpy()
        mean, std = best_metrics.mean(), best_metrics.std()
        final_metrics.append(f'{mean:.4f} (±{std:.2f})')
        
    final_metrics_df = final_metrics_df.append(pd.DataFrame([final_metrics], columns=cols, index=(fm_col,)))

final_metrics_df

,acc-all,ppv-all,ppv-covid,ppv-pneumonia,ppv-normal,recall-all,recall-covid,recall-pneumonia,recall-normal,f1-all,f1-covid,f1-pneumonia,f1-normal
fixaug_nl100_2-nl100-m1-b16-r0,0.8622 (±0.02),0.8657 (±0.02),0.9214 (±0.01),0.8357 (±0.05),0.8398 (±0.01),0.8622 (±0.02),0.8567 (±0.04),0.8733 (±0.02),0.8567 (±0.03),0.8626 (±0.02),0.8871 (±0.02),0.8527 (±0.02),0.8480 (±0.02)
fixaug_nl100_2-nl100-m1-b16-r1,0.8689 (±0.01),0.8713 (±0.01),0.9201 (±0.00),0.8471 (±0.02),0.8467 (±0.01),0.8689 (±0.01),0.8433 (±0.02),0.8800 (±0.02),0.8833 (±0.02),0.8690 (±0.01),0.8799 (±0.01),0.8627 (±0.00),0.8645 (±0.01)
fixaug_nl100_2-nl100-m2-b16-r0,0.8667 (±0.01),0.8696 (±0.01),0.9179 (±0.02),0.8688 (±0.02),0.8222 (±0.01),0.8667 (±0.01),0.8500 (±0.02),0.8567 (±0.03),0.8933 (±0.02),0.8669 (±0.01),0.8824 (±0.02),0.8622 (±0.02),0.8562 (±0.01)
fixaug_nl100_2-nl100-m2-b16-r1,0.8656 (±0.01),0.8706 (±0.01),0.9194 (±0.01),0.8625 (±0.04),0.8300 (±0.04),0.8656 (±0.01),0.8333 (±0.02),0.8733 (±0.03),0.8900 (±0.04),0.8659 (±0.01),0.8739 (±0.01),0.8664 (±0.01),0.8573 (±0.00)
fixaug_nl100_2-nl100-m3-b16-r0,0.8700 (±0.01),0.8734 (±0.01),0.9181 (±0.04),0.8717 (±0.03),0.8303 (±0.01),0.8700 (±0.01),0.8633 (±0.04),0.8667 (±0.03),0.8800 (±0.01),0.8704 (±0.01),0.8884 (±0.02),0.8682 (±0.01),0.8544 (±0.01)
fixaug_nl100_2-nl100-m3-b16-r1,0.8678 (±0.02),0.8766 (±0.02),0.9526 (±0.01),0.8689 (±0.06),0.8081 (±0.01),0.8678 (±0.02),0.8100 (±0.04),0.8700 (±0.01),0.9233 (±0.03),0.8683 (±0.02),0.8753 (±0.03),0.8682 (±0.03),0.8614 (±0.01)


In [103]:
# from itertools import permutations
# list(permutations([9, 14, 3]))

[(9, 14, 3), (9, 3, 14), (14, 9, 3), (14, 3, 9), (3, 9, 14), (3, 14, 9)]

In [58]:
final_metrics_df.to_csv(f'{out_dir}/{out_fname}')

In [4]:
from collections import OrderedDict, Callable

class DefaultOrderedDict(OrderedDict):
    # Source: http://stackoverflow.com/a/6190500/562769
    def __init__(self, default_factory=None, *a, **kw):
        if (default_factory is not None and
           not isinstance(default_factory, Callable)):
            raise TypeError('first argument must be callable')
        OrderedDict.__init__(self, *a, **kw)
        self.default_factory = default_factory

    def __getitem__(self, key):
        try:
            return OrderedDict.__getitem__(self, key)
        except KeyError:
            return self.__missing__(key)

    def __missing__(self, key):
        if self.default_factory is None:
            raise KeyError(key)
        self[key] = value = self.default_factory()
        return value

    def __reduce__(self):
        if self.default_factory is None:
            args = tuple()
        else:
            args = self.default_factory,
        return type(self), args, None, None, self.items()

    def copy(self):
        return self.__copy__()

    def __copy__(self):
        return type(self)(self.default_factory, self)

    def __deepcopy__(self, memo):
        import copy
        return type(self)(self.default_factory,
                          copy.deepcopy(self.items()))

    def __repr__(self):
        return 'OrderedDefaultDict(%s, %s)' % (self.default_factory,
                                               OrderedDict.__repr__(self))